# Нейронные сети

## Полносвязные нейронные сети (FCNN)

### Одиночный нейрон (Перцептрон)

Одиночный нейрон вычисляет взвешенную сумму его входов, добавляет свободный коэффициент (bias/смещение), а затем пропускает результат через функцию активации:

$$y = \phi\bigl(\mathbf{w}^\top \mathbf{x} + b\bigr)$$
* Веса $\mathbf{w}$: взвешиватют каждый вход
* Смещение $b$: смещает порог активации
* Активация $\phi(\cdot)$: вводит нелинейность, позволяя сети учить сложные зависимости

Популярные функции активации:
* Пороговая: $\phi(z)=\begin{cases}1,&z\ge0\\0,&z<0\end{cases}$
* Sigmoid: $\phi(z)=\frac{1}{1 + e^{-z}}$, outputs in (0,1)
* Tanh: $\phi(z)=\tanh(z)$, outputs in (−1,1)
* ReLU: $\phi(z)=\max(0,z)$, sparse activations, fast convergence
* Leaky ReLU: $\phi(z)=\max(\alpha z, z)$, alleviates “dead” neurons

#### PyTorch

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class SingleNeuron(nn.Module):
    def __init__(self, in_features, activation='relu'):
        super().__init__()
        # Linear layer: one output neuron
        self.linear = nn.Linear(in_features, 1)
        # Select activation
        activations = {
            'step': lambda x: (x >= 0).float(),
            'sigmoid': nn.Sigmoid(),
            'tanh': nn.Tanh(),
            'relu': nn.ReLU(),
            'leaky_relu': nn.LeakyReLU(negative_slope=0.01)
        }
        self.act = activations[activation]

    def forward(self, x):
        z = self.linear(x)            # compute w·x + b
        return self.act(z)            # apply chosen activation

In [ ]:
# Example usage:
model = SingleNeuron(in_features=10, activation='sigmoid')
output = model(torch.randn(5, 10))  # batch of 5 samples

In [ ]:
print(output)

#### TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
def build_single_neuron(input_dim, activation='relu'):
    model = models.Sequential([
        # Dense layer with 1 unit, bias included
        layers.Dense(1,
                     activation=activation,
                     input_shape=(input_dim,))
    ])
    return model

In [ ]:
# Example usage:
model = build_single_neuron(input_dim=10, activation='tanh')
preds = model(tf.random.normal((5, 10)))  # batch of 5 samples

In [ ]:
preds

### Простая полносвязная нейронная сеть

Полносвязная нейронная сеть состоит из входного слоя, одного или нескольких скрытых слоев (каждый нейрон соединен с каждым нейроном в соседних слоях) и выходного слоя. Слоистая структура с нелинейными активациями позволяет сети могут аппроксимировать сложные функции.
* Для скрытых слоев выбирают нелинейную функцию активации (например, ReLU).
* Выходной слой и функция потерь зависят от задачи:
    * Классификация (двоичная/многоклассовая): Softmax (многоклассовая) или Sigmoid (двоичная); функция потерь -- кросс-энтропия
    * Регрессия: конечная(ыe) функция(и) активации линейная(ые); функция потерь = MSE

#### PyTorch

In [ ]:
# ── Classification ──
class FCNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        # Sequential: Linear → ReLU → Linear (logits)
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),                       # hidden activation
            nn.Linear(hidden_dim, num_classes)
            # no Softmax here: CrossEntropyLoss expects raw logits
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
# Example usage:
model = FCNNClassifier(input_dim=20, hidden_dim=50, num_classes=3)
logits = model(torch.randn(8, 20))                   # batch of 8
loss = nn.CrossEntropyLoss()(logits, torch.randint(0, 3, (8,)))

In [ ]:
# ── Regression ──
class FCNNRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        # Sequential: Linear → ReLU → Linear (output)
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),                       # hidden activation
            nn.Linear(hidden_dim, 1)         # single continuous output
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
# Example usage:
model = FCNNRegressor(input_dim=20, hidden_dim=50)
preds = model(torch.randn(8, 20))                   # batch of 8
loss = nn.MSELoss()(preds, torch.randn(8, 1))

#### TensorFlow

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# ── Classification ──
def build_fcnn_classifier(input_dim, hidden_units, num_classes):
    model = Sequential([
        Dense(hidden_units, activation='relu', input_shape=(input_dim,)),
        Dense(num_classes, activation='softmax')  # multi-class probabilities
    ])
    # Compile with optimizer and categorical cross-entropy
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
# Example usage:
clf = build_fcnn_classifier(input_dim=20, hidden_units=50, num_classes=3)
clf.summary()

In [ ]:
# ── Regression ──
def build_fcnn_regressor(input_dim, hidden_units):
    model = Sequential([
        Dense(hidden_units, activation='relu', input_shape=(input_dim,)),
        Dense(1)  # linear activation by default
    ])
    # Compile with optimizer and MSE loss
    model.compile(optimizer='adam',
                  loss='mean_squared_error')
    return model


In [ ]:
# Example usage:
reg = build_fcnn_regressor(input_dim=20, hidden_units=50)
reg.summary()

### Процесс обучения FCNN. Метод обратного распространения ошибки. Градиентный спуск и его вариации. Стратегии инициализации параметров. Эпохи. Скорость обучения

#### Общий процесс обучения
1.	Прямой проход
    * Входной батч → сеть → выход
2.	Вычисление потерь
    * Вычисление расхождения между прогнозами и таргетами с помощью функции потерь (например, кросс-энтропии или MSE)
3. Обратный проход (обратное распространение)
    * Вычисление градиентов потерь по каждому параметру с помощью chain rule
4.	Обновление параметров (градиентный спуск)
    * Корректировка весов/смещений с помощью градиентов и скорости обучения
5.	Повторить предыдущие 4 пункта на каждом батче и эпохе, пока не сойдется или не остановимся по другим критериям





### Метод обратного распространения ошибки
* Цель: вычислить $\frac{\partial L}{\partial w_{ij}}$ для каждого $w_{ij}$.
* Chain rule (цепочки производных):
$\frac{\partial L}{\partial w_{ij}} = \frac{\partial L}{\partial z_j}\;\frac{\partial z_j}{\partial w_{ij}}
\quad\text{где}\quad z_j = \sum_i w_{ij} a_i + b_j$

Процесс:

1.	На выходном слое, $\delta^{(L)} = \nabla_{a} L \odot \phi{\prime}(z)$

2.	На каждом предыдущем слое $l$: $\delta^{(l)} = (W^{(l+1)})^\top \delta^{(l+1)} \odot \phi{\prime}(z^{(l)})$

3.	Градиенты: $\nabla_{W^{(l)}} L = \delta^{(l)} (a^{(l-1)})^\top,\quad \nabla_{b^{(l)}} L = \delta^{(l)}$





### Вариации градиентного спуска
* Пакетный (Batch GD)
    * Использует всю обучающую выборку для вычисления градиентов на каждом шаге
    * Стабильный, но медленный и затратный по памяти
* Стохастический (SGD)
    * Прогон по одному примеру
    * Зашумленный обновления весов позволяют избежать локальных минимумов
    * Процесс обучения не стабилен и сильно зависит от кол-ва шума в исходных данных
* Мини-пакетный (Mini batch GD)
    * Компромисс: использование небольших партий (32-256 образцов) исходной выборки
    * Распространен на практике для повышения эффективности и стабильности





### Стратегии инициализации параметров
* Инициализация нулями
    * Плохо: симметричные веса выучивают одни и те же зависимости
* Нормальное/равномерное равномерное
    * Ломает симметрию, но на практике важна дисперсия
* Xavier/Glorot
$$\mathrm{Var}(w) = \frac{2}{n_\text{in}+n_\text{out}}$$
    хорошо для tanh/sigmoid
* He (Kaiming)
$$\mathrm{Var}(w) = \frac{2}{n_\text{in}}$$
    хорошо для ReLU






### Эпохи и скорость обучения
* Эпоха: один полный прогон обучающей выборки
* Скорость обучения (Learning rate)
    * Отвечает за размер шага градиентного спуска: слишком большой → рискуем пропустить минимум; слишком маленький → медленно
    * Обычно скорость обучения уменьшают с увеличением количества пройденных эпох

#### PyTorch

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Assume MyDataset yields (features, labels)
train_ds = MyDataset(...)
loader = DataLoader(train_ds, batch_size=64, shuffle=True)

model = FCNNClassifier(input_dim=..., hidden_dim=..., num_classes=...)

In [ ]:

# Xavier init for all Linear layers
for m in model.modules():
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

criterion = nn.CrossEntropyLoss()            # classification loss
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.01,                                   # initial learning rate
    momentum=0.9
)

num_epochs = 20
for epoch in range(1, num_epochs+1):
    model.train()                             # set training mode
    total_loss = 0.0
    for X, y in loader:                       # mini-batch loop
        optimizer.zero_grad()                 # zero gradients
        logits = model(X)                     # forward pass
        loss = criterion(logits, y)           # compute loss
        loss.backward()                       # backpropagation
        optimizer.step()                      # update parameters
        total_loss += loss.item() * X.size(0)
    avg_loss = total_loss / len(train_ds)
    print(f"Epoch {epoch:2d}, Loss: {avg_loss:.4f}")

#### TensorFlow

In [ ]:
import tensorflow as tf

# Prepare dataset
train_ds = (tf.data.Dataset.from_tensor_slices((X_train, y_train))
            .shuffle(1000)
            .batch(64))

model = build_fcnn_classifier(input_dim=..., hidden_units=..., num_classes=...)
# He initialization applied by default for 'relu' Dense layers in Keras

In [ ]:
# Compile with SGD optimizer
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train for a fixed number of epochs
history = model.fit(
    train_ds,
    epochs=20,
    verbose=2
)

### Обучение глубоких FCNN. Проблема исчезающего градиента (vanishing gradient problem). Проблема взрывающегося градиента (exploding gradient problem). Методы регуляризации. Методы увеличения стабильности сети.

#### Исчезающие и Взрывающиеся Градиента
* Проблема исчезающего градиента возникает, когда частные производные в методе обратного распространения уменьшаются экспоненциально при переходе от слоя к слою - это характерно для sigmoid/tanh ФА - и слои, расположенные ближе к началу, обучаются очень медленно.
* Проблема взрывающегося градиента возникает, когда производные очень быстро растут, что приводит к нестабильному обновлению и переполнению параметров.

#### Методы регуляризации
* L2 weight decay: добавка $\lambda\|\mathbf W\|^2$ к loss функции, препятствует весам быть большими.
* Dropout: случайным образом зануляет активации при обучении для предотвращения ко-адаптации.
* Early stopping (ранняя остановка): завершает процесс обучения, когда loss не уменьшается какое-то время

#### Методы увеличения стабильности сети
* Качественная инициализация весов (Xavier для tanh/sigmoid, He для ReLU) для поддержки стабильности дисперсии.
* Batch Normalization (или LayerNorm): нормализует входы слоя (mean=0, var=1), сглаживая оптимизируемую поверхность.
* Gradient clipping (подрезка градиентов): зафиксировать рамки градиентов для предотвращения проблемы взрывающихся градиентов.
* Residual connections: дополнительные связи между слоями, чтобы градиенты могли проходить беспрепятственно.
* Использование ReLU (или его вариантов) для предотвращения исчезновения градиентов.

#### PyTorch

In [ ]:
class DeepFCNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, dropout_p=0.5):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h in hidden_dims:
            # Linear → BatchNorm → ReLU → Dropout
            layers += [
                nn.Linear(prev_dim, h),
                nn.BatchNorm1d(h),             # stabilizes activations
                nn.ReLU(inplace=True),         # non-saturating activation
                nn.Dropout(p=dropout_p)        # regularization
            ]
            prev_dim = h
        layers.append(nn.Linear(prev_dim, num_classes))  # output layer
        self.net = nn.Sequential(*layers)

        # He init for all Linear layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.net(x)

In [ ]:
# ── Training with gradient clipping ──
model = DeepFCNN(input_dim=100,
                 hidden_dims=[256, 256, 128],
                 num_classes=10,
                 dropout_p=0.5)

optimizer = torch.optim.AdamW(model.parameters(),
                              lr=1e-3,         # learning rate
                              weight_decay=1e-4)  # L2 regularization

criterion = nn.CrossEntropyLoss()

for epoch in range(1, 51):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        logits = model(X_batch)              # forward
        loss = criterion(logits, y_batch)    # compute loss
        loss.backward()                      # backpropagate
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # clip gradients to max norm=1.0
        optimizer.step()                     # update
    # ... validation & early stopping ...

#### TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

def build_deep_fcnn(input_dim, hidden_units, num_classes, dropout_p=0.5):
    model = Sequential()
    for h in hidden_units:
        model.add(Dense(
            h,
            activation=None,
            kernel_initializer='he_normal',      # He init for ReLU
            kernel_regularizer=l2(1e-4),         # L2 regularization
            input_shape=(input_dim,) if model.layers==[] else None
        ))
        model.add(BatchNormalization())          # stabilize activations
        model.add(tf.keras.layers.ReLU())        # non-saturating activation
        model.add(Dropout(dropout_p))            # regularization

    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
model = build_deep_fcnn(input_dim=100,
                        hidden_units=[256, 256, 128],
                        num_classes=10,
                        dropout_p=0.5)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3,
                                     clipnorm=1.0)   # gradient clipping

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model.fit(train_ds,
          epochs=50,
          validation_data=val_ds,
          callbacks=[early_stop])

### Оптимизаторы

1. Stochastic Gradient Descent (SGD)
    * Как обновляется: $w_{t+1} = w_t - \eta \,\nabla_w L(w_t)$
    * Простой, не требует много памяти
	* Медленно сходится, чувствителен к сложным поверхностям
	* Гиперпараметры: learning rate
2. SGD + Momentum
    * Мотивация: накапливать вектор скорости для сглаживания обновлений и ускорения при обновлениях в одном направлении.
	* Как обновляется: $v_{t+1} = \mu\,v_t - \eta\,\nabla_w L(w_t),\quad w_{t+1} = w_t + v_{t+1}$
	* Быстрее сходится, меньше колеблется
	* Добавляется еще один гиперпараметр (momentum $μ$)
3. Nesterov Accelerated Gradient (NAG)
	* Вычисляет градиент в приблизительной будущей точке
	* Как обновляется: $v_{t+1} = \mu\,v_t - \eta\,\nabla_w L(w_t + \mu\,v_t),\quad w_{t+1} = w_t + v_{t+1}$
	* Обычно быстрее чем ванильный моментум
	* Те же гиперпараметры, что и у моментума
4. AdaGrad
	* Идея: адаптировать скорость обучения каждого параметра на основе их прошлых градиентов.
	* Как обновляется: $r_t = r_{t-1} + (\nabla_w L(w_t))^2,\quad w_{t+1} = w_t - \frac{\eta}{\sqrt{r_t + \epsilon}}\,\nabla_w L(w_t)$
	* Круто для разреженных фичей
	* Скорость обучения уменьшается слишком агрессивно
5. RMSProp
	* Допиленный адаград: использует экспоненциальное скользящее среднее квадратов градиентов.
	* Как обновляется: $r_t = \rho\,r_{t-1} + (1-\rho)(\nabla_w L)^2,\quad w_{t+1} = w_t - \frac{\eta}{\sqrt{r_t + \epsilon}}\,\nabla_w L$
	* Более стабильный learnin rate
	* Гиперпараметры: decay rate ρ (default 0.9)
6. AdaDelta
	* Расширение RMSProp: дополнительно скейлит обновления без необходимости глобального параметра скорости обучения
	* Нет необходимости настраивать learning rate
	* Более вычислительно сложный, по сравнению с RMSProp
7. Adam
	* Комбинирует идеи моментума и RMSProp
	* Как обновляется: $m_t = \beta_1 m_{t-1} + (1-\beta_1)\nabla,\quad v_t = \beta_2 v_{t-1} + (1-\beta_2)\nabla^2$
	  смещение
$$\hat m_t = m_t/(1-\beta_1^t),\,\hat v_t = v_t/(1-\beta_2^t)$$
$$w_{t+1} = w_t - \eta\,\frac{\hat m_t}{\sqrt{\hat v_t} + \epsilon}$$
    * Быстро сходится
	* Гиперпараметры: β₁≈0.9, β₂≈0.999
8. AdamW
	* Исключает затухание весов из оценки момента в Adam
    * Более сильная L2 регуляризация

#### PyTorch

In [ ]:
# PyTorch optimizer examples

import torch.optim as optim

# 1. SGD
optim.SGD(model.parameters(),
          lr=0.01)                                # base SGD

# 2. SGD + Momentum
optim.SGD(model.parameters(),
          lr=0.01,
          momentum=0.9)                          # add momentum µ=0.9

# 3. Nesterov
optim.SGD(model.parameters(),
          lr=0.01,
          momentum=0.9,
          nesterov=True)                        # NAG

# 4. AdaGrad
optim.Adagrad(model.parameters(),
              lr=0.01,
              eps=1e-8)                          # per‐param adaptivity

# 5. RMSProp
optim.RMSprop(model.parameters(),
              lr=0.001,
              alpha=0.9,                         # decay rate ρ
              eps=1e-8)

# 6. AdaDelta
optim.Adadelta(model.parameters(),
               lr=1.0,                             # often default
               rho=0.95)

# 7. Adam
optim.Adam(model.parameters(),
           lr=0.001,
           betas=(0.9, 0.999),                   # β₁,β₂
           eps=1e-8)

# 8. AdamW
optim.AdamW(model.parameters(),
            lr=0.001,
            betas=(0.9, 0.999),
            weight_decay=1e-2)                  # decoupled L2

#### TensorFlow

In [ ]:
# TensorFlow (Keras) optimizer examples

import tensorflow as tf

# 1. SGD
tf.keras.optimizers.SGD(learning_rate=0.01)

# 2. SGD + Momentum
tf.keras.optimizers.SGD(learning_rate=0.01,
                        momentum=0.9)

# 3. Nesterov
tf.keras.optimizers.SGD(learning_rate=0.01,
                        momentum=0.9,
                        nesterov=True)

# 4. AdaGrad
tf.keras.optimizers.Adagrad(learning_rate=0.01,
                            epsilon=1e-8)

# 5. RMSProp
tf.keras.optimizers.RMSprop(learning_rate=0.001,
                            rho=0.9,
                            epsilon=1e-8)

# 6. AdaDelta
tf.keras.optimizers.Adadelta(learning_rate=1.0,
                             rho=0.95)

# 7. Adam
tf.keras.optimizers.Adam(learning_rate=0.001,
                         beta_1=0.9,
                         beta_2=0.999,
                         epsilon=1e-8)

# 8. AdamW
tf.keras.optimizers.experimental.AdamW(learning_rate=0.001,
                                       weight_decay=1e-2)

### Мониторинг процесса обучения на валидационной выборке

Зачем?
* Обнаружение переобучения: лосс на валидационной выборке растет, в то время когда на обучающей падает
* Ранняя остановка: остановиться, когда лосс на валидационной перестает падать
* Настройка гиперпараметров: выбор скорости обучения, архитектуры модели и регуляризации

#### PyTorch

In [ ]:
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt

# Assume model, criterion, optimizer defined; train_loader & val_loader ready
history = {'train_loss':[], 'val_loss':[], 'val_acc':[]}

for epoch in range(1, num_epochs+1):
    # ——— Training ———
    model.train()
    running_loss = 0.0
    for X,y in train_loader:
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)
    history['train_loss'].append(running_loss/len(train_loader.dataset))

    # ——— Validation ———
    model.eval()
    val_loss, all_preds, all_targets = 0.0, [], []
    with torch.no_grad():
        for Xv, yv in val_loader:
            logits = model(Xv)
            val_loss += criterion(logits, yv).item() * Xv.size(0)
            preds = torch.argmax(logits, dim=1)
            all_preds.append(preds.cpu())
            all_targets.append(yv.cpu())
    val_loss /= len(val_loader.dataset)
    history['val_loss'].append(val_loss)

    # Compute validation accuracy
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    acc = accuracy_score(all_targets, all_preds)  # Or another metrics
    history['val_acc'].append(acc)

    print(f"Epoch {epoch}: train_loss={history['train_loss'][-1]:.4f}, "
          f"val_loss={val_loss:.4f}, val_acc={acc:.4f}")

# ——— Plotting ———
epochs = range(1, num_epochs+1)
plt.figure()                                      # Loss curves
plt.plot(epochs, history['train_loss'], label='Train')
plt.plot(epochs, history['val_loss'],   label='Val')
plt.title('Loss vs. Epoch'); plt.xlabel('Epoch'); plt.ylabel('Loss')
plt.legend(); plt.show()

plt.figure()                                      # Accuracy curve
plt.plot(epochs, history['val_acc'], label='Val Acc')
plt.title('Val Accuracy vs. Epoch'); plt.xlabel('Epoch'); plt.ylabel('Accuracy')
plt.legend(); plt.show()

#### TensorFlow

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, confusion_matrix

# Assume model built and compiled with metrics=['accuracy']
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=num_epochs,
    verbose=2
)

# ——— Plotting ———
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss vs. Epoch'); plt.xlabel('Epoch'); plt.ylabel('Loss')
plt.legend(); plt.show()

plt.figure()
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy vs. Epoch'); plt.xlabel('Epoch'); plt.ylabel('Accuracy')
plt.legend(); plt.show()

# ——— Additional Metrics ———
# Compute ROC AUC on validation set
y_true, y_pred_probs = [], []
for Xv, yv in val_ds:
    y_true.append(yv.numpy())
    y_pred_probs.append(model.predict(Xv))

y_true = np.concatenate(y_true)
y_pred_probs = np.concatenate(y_pred_probs)
# If multiclass, average='macro', multi_class='ovr'
print("ROC AUC:", roc_auc_score(y_true, y_pred_probs))

# Confusion matrix
y_pred = np.argmax(y_pred_probs, axis=1)
cm = confusion_matrix(y_true, y_pred)
plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix'); plt.xlabel('Predicted'); plt.ylabel('True')
plt.colorbar(); plt.show()

Также можно использовать тулзу под названием TensorBoard
PyTorch: torch.utils.tensorboard.SummaryWriter
Keras: передать колбэк TensorBoard в model.fit(...) для автоматического логгирования всего что вам хочется.

### Инференс и использования FCNN

#### Режим инференса и подготовка модели
1. Включить режим инференса, чтобы слои Dropout, BatchNorn и другие вели себя детерминистично:
	* В PyTorch: `model.eval()`
	* В Keras: модели всегда в готовом для инференса состоянии после вызова model.compile() или load_model()
2. Выключить отслеживание градиентов для ускорения инференса и сохранения памяти:
	* В PyTorch: обернуть `forward()` в `with torch.no_grad():`
	* В Keras: `.predict()` делает это автоматически


#### Подготовка входных данных для инференса
1. Препроцессить фичи **точно так же** как и при обучении

#### PyTorch

In [ ]:
import torch

# 0. Save only the state dict (recommended)
torch.save(model.state_dict(), 'fcnn_classifier.pth')

# 1. Load model architecture & weights
model = FCNNClassifier(input_dim=20, hidden_dim=50, num_classes=3)
state = torch.load('fcnn_classifier.pth', map_location='cpu')  # or 'cuda'
model.load_state_dict(state)                                   # load trained parameters
model.eval()                                                   # set eval mode

# 2. Prepare new data
# Suppose new_sample is a NumPy array of shape (20,)
import numpy as np
new_sample = np.random.rand(20).astype(np.float32)             # dummy data
# Normalize same way as training data
mean, std = 0.5, 0.2                                           # example values
new_sample = (new_sample - mean) / std

# 3. Convert to tensor & batch
input_tensor = torch.from_numpy(new_sample).unsqueeze(0)       # shape (1,20)

# 4. Forward pass
with torch.no_grad():
    logits = model(input_tensor)                               # raw scores
    probs  = torch.softmax(logits, dim=1)                      # probabilities
    class_idx = torch.argmax(probs, dim=1).item()              # predicted class

print(f"Predicted class: {class_idx}, Probabilities: {probs.squeeze().tolist()}")

#### TensorFlow

In [ ]:
import numpy as np
import tensorflow as tf

# 0. Save the full model (architecture + weights + optimizer state)
model.save('fcnn_model')

# 1. Load the saved model
model = tf.keras.models.load_model('fcnn_model')

# 2. Prepare new data
new_sample = np.random.rand(20).astype('float32')              # dummy input
# Normalize with training mean/std
mean, std = 0.5, 0.2                                           # example
new_sample = (new_sample - mean) / std

# 3. Batch dimension
input_batch = np.expand_dims(new_sample, axis=0)               # shape (1,20)

# 4. Predict
probs = model.predict(input_batch)                             # shape (1, num_classes)
class_idx = np.argmax(probs, axis=1)[0]                        # predicted class

print(f"Predicted class: {class_idx}, Probabilities: {probs[0].tolist()}")

Если у вас есть ускоритель

В PyTorch, перенести модель и данные на GPU через `.to('cuda')` перед инференсом
В Keras, большие батчи автоматом используют все доступные GPU

## Сверточные нейронные сети

### Основы

CNN обрабатывает структурированные данные (чаще всего изображения и видео), заменяя полносвязанные слои сверточными, использующими пространственную локальность и совместное использование параметров.

1. Сверточный слой
	* Применяет обучаемые ядра (фильтры) к локальным рецептивным полям
	* Создает карты признаков: каждый фильтр обнаруживает определенный паттерн (края, текстуры)
	* Параметр `stride` управляет размером шага; параметр `padding` сохраняет пространственные размеры
2. Активация
	* Нелинейная функция (ReLU, LeakyReLU) применяется поэлементно к картам признаков
3. Слой Pooling
	* Уменьшение размерности признакового пространства с помощью max/avg.
	* Уменьшает вычислительную сложность
4. Слоистая структура
	* Более ранние слои фиксируют низкоуровневые характеристики (края); более глубокие слои фиксируют высокоуровневые концепты (отдельные объекты)
5. Выпрямление и FCNN
	* Итоговые карты признаков выпрямляются и проходят через полносвязные слои для задач классификации, регрессии или других

#### PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # Convolution: in_channels=3 (RGB), out_channels=16, kernel_size=3×3
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        # Convolution: 16 → 32 feature maps
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        # Max pooling with 2×2 window
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected: flatten 32×8×8 → 128 neurons
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        # Output layer: 128 → num_classes
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # conv1 → ReLU → pool ⇒ output size: 16×16×16
        x = self.pool(F.relu(self.conv2(x)))  # conv2 → ReLU → pool ⇒ output size: 32×8×8
        x = x.view(x.size(0), -1)             # flatten feature maps
        x = F.relu(self.fc1(x))               # fully connected + ReLU
        x = self.fc2(x)                       # final logits
        return x

In [ ]:
# Example usage:
model = SimpleCNN(num_classes=10)
imgs = torch.randn(4, 3, 32, 32)       # batch of 4 RGB 32×32 images
logits = model(imgs)

#### TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_simple_cnn(input_shape=(32, 32, 3), num_classes=10):
    model = models.Sequential([
        # Conv2D: 16 filters of size 3×3, padding='same'
        layers.Conv2D(16, (3, 3), padding='same', activation='relu',
                      input_shape=input_shape),
        # MaxPooling2D: downsample by factor of 2
        layers.MaxPooling2D((2, 2)),
        # Conv2D: 32 filters of size 3×3
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D((2, 2)),
        # Flatten feature maps to vector
        layers.Flatten(),
        # Dense hidden layer with 128 units
        layers.Dense(128, activation='relu'),
        # Output layer with softmax for classification
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
# Example usage:
model = build_simple_cnn((32, 32, 3), 10)
model.summary()

### Pooling


Cлои Pooling выполняют понижающую дискретизацию карт признаков, обобщая локальные окрестности, уменьшая пространственные размеры, сложность вычислений и (в некоторой степени) переобучение.

| Pooling Type | Operation                                          | Use Case & Properties                                                                                      |
| ------------ |----------------------------------------------------|------------------------------------------------------------------------------------------------------------|
| Max Pooling | $\max$ over each $k\times k$ window                | Captures strongest activation; preserves salient features (edges, corners); common default.                |
| Average Pooling | Mean over each $k\times k$ window                  | Smooths activations; retains background context; sometimes used in regression or heatmap tasks.            |
| Global Average/Max | Pool over entire spatial dims -> scalar per channel | Aggressive downsampling to $1\times 1$; often before FC layers; reduces parameters; used in classification. |
| Adaptive Pooling | Output to a specified spatial size | Guarantees fixed output regardless of input size; useful for variable-resolution inputs. |


#### PyTorch

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# 1. Local Max Pooling (2×2 window, stride 2)
max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

# 2. Local Average Pooling (2×2 window, stride 2)
avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)

# 3. Global Average Pooling → output size = (1,1)
global_avg = nn.AdaptiveAvgPool2d((1, 1))

# 4. Global Max Pooling → output size = (1,1)
global_max = nn.AdaptiveMaxPool2d((1, 1))

# Example forward usage in a nn.Module:
class PoolingDemo(nn.Module):
    def forward(self, x):
        x1 = max_pool(x)              # strong activations
        x2 = avg_pool(x)              # smoothed activations
        x3 = global_avg(x).view(x.size(0), -1)  # flatten to (batch, channels)
        return x1, x2, x3

#### TensorFlow

In [ ]:
from tensorflow.keras import layers, models

def build_pooling_demo(input_shape=(32,32,3)):
    inputs = layers.Input(shape=input_shape)

    # 1. Local Max Pooling
    x1 = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(inputs)

    # 2. Local Average Pooling
    x2 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2))(inputs)

    # 3. Global Average Pooling → (batch_size, channels)
    x3 = layers.GlobalAveragePooling2D()(inputs)

    # 4. Global Max Pooling → (batch_size, channels)
    x4 = layers.GlobalMaxPooling2D()(inputs)

    return models.Model(inputs, [x1, x2, x3, x4])

# Example instantiation:
model = build_pooling_demo()
outputs = model(tf.random.normal((4,32,32,3)))

### Работа с изображениями с разными разрешениями. Padding

#### Зачем и когда?
1. Сохранить соотношения сторон: Вместо того чтобы растягивать или сжимать изображение, можно добавить padding, чтобы оно соответствовало нужной форме (например, 224×224).
2. Предотвратить потерю информации: Кропая изображение можно потерять нужные детали. Паддинг позволяет этого избежать
3. Эффективность батчинга: Сети ожидают одинаковые размеры матриц, паддинг позволяет батчить разные изображения вместе.

| Тип       | Описание                               | Комментарий                                |
|-----------|----------------------------------------|--------------------------------------------|
| Zero      | заполнить нулями (черный цвет границы) | может добавить дополнительные края (плохо) |
| Reflect   | отразить изображение за границей       | сохраняет текстуры и края                  |
| Replicate | повторять последний пиксель на границе | менее жестко, нежели нули                  |
| Constant  | заполнить константным цветом           | белый цвет для сканов документов           |

#### PyTorch

In [ ]:
from torchvision import transforms
from PIL import Image

# 1. Compute padding amounts for a given image to make it square (e.g. 224×224)
def pad_to_square(img, fill=0):
    w, h = img.size
    max_side = max(w, h)
    pad_left = (max_side - w) // 2
    pad_right = max_side - w - pad_left
    pad_top = (max_side - h) // 2
    pad_bottom = max_side - h - pad_top
    # transforms.Pad expects (left, top, right, bottom)
    return transforms.Pad((pad_left, pad_top, pad_right, pad_bottom), fill=fill)(img)

def reflect_pad_to_square(img):
    w, h = img.size
    max_side = max(w, h)
    pads = [(max_side - w) // 2, (max_side - h) // 2,
            max_side - w - (max_side - w)//2, max_side - h - (max_side - h)//2]
    return F.pad(img, pads, padding_mode='reflect')

# 2. Compose a transform: pad → resize → to tensor → normalize
transform = transforms.Compose([
    transforms.Lambda(lambda img: pad_to_square(img, fill=128)),  # gray padding. Or use reflect_pad_to_square
    transforms.Resize((224, 224)),                                # final size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

# Usage in a Dataset
class MyImageDataset(torch.utils.data.Dataset):
    def __init__(self, paths, transform=None):
        self.paths = paths
        self.transform = transform
    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img
    def __len__(self):
        return len(self.paths)

#### TensorFlow

In [ ]:
import tensorflow as tf

def pad_to_square_tf(image, pad_value=0):
    # image: [H, W, C]
    shape = tf.shape(image)[:2]
    h, w = shape[0], shape[1]
    max_side = tf.maximum(h, w)
    pad_h = max_side - h
    pad_w = max_side - w
    pad_top = pad_h // 2
    pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2
    pad_right = pad_w - pad_left
    # paddings: [[top, bottom], [left, right], [0,0]]
    paddings = [[pad_top, pad_bottom], [pad_left, pad_right], [0, 0]]
    return tf.pad(image, paddings, constant_values=pad_value)

# Build a preprocessing layer
def build_preprocessing_layer(target_size=(224,224), pad_value=128):
    return tf.keras.Sequential([
        tf.keras.layers.Lambda(lambda img: pad_to_square_tf(img, pad_value)),
        tf.keras.layers.Resizing(target_size[0], target_size[1]),
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.Normalization(mean=[0.485,0.456,0.406],
                                      variance=[0.229**2,0.224**2,0.225**2])
    ])

# Usage in tf.data pipeline
preproc = build_preprocessing_layer()
dataset = tf.data.Dataset.from_tensor_slices(list_of_paths)
dataset = dataset.map(lambda path: tf.io.read_file(path)
                                 .pipe(tf.image.decode_jpeg, channels=3)
                                 .pipe(lambda img: preproc(img)))

### Регуляризация в CNN


1.	Weight Decay (L2 Regularization)
	*	Штрафует веса, если они слишком большие, добавляя $\lambda \|\mathbf W\|^2$ к лосс функции
	*	Уже написано за вас в оптимизаторах (`weight_decay`, `kernel_regularizer=l2`)
2. Spatial Dropout / DropBlock
	* SpatialDropout зануляет карты признаков (каналы) целиком для увеличения стабильности
	* DropBlock случайным образом маскирует регионы на картах признаков
3. Batch Normalization
	* То же самое, что и в FCNN, но 2D
4. Аугментация данных
	* Геометрическая: отражения, кропы, повороты
	* Фотометрическая: изменение цветов, изменения контраста
5. Label Smoothing
	* Смягчает one-hot таргеты (до 0.9/0.1) для уменьшения "слишком уверенных" предсказаний
6. Early Stopping
    * То же самое, что и в FCNN

#### PyTorch

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
import torchvision.transforms as T
from torchvision.models import resnet18

# 1. Data augmentation pipeline
train_transforms = T.Compose([
    T.RandomResizedCrop(224),          # random crop & resize
    T.RandomHorizontalFlip(),          # flip left-right
    T.ColorJitter(brightness=0.2,
                  contrast=0.2,
                  saturation=0.2),     # photometric
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406],
                std=[0.229,0.224,0.225])
])

# 2. Model with BatchNorm and SpatialDropout
class CNNWithReg(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.backbone = resnet18(pretrained=False)
        # Replace final layer and add dropout
        self.backbone.fc = nn.Sequential(
            nn.Dropout(p=0.5),      # dropout before FC
            nn.Linear(self.backbone.fc.in_features, num_classes)
        )
    def forward(self, x):
        return self.backbone(x)

model = CNNWithReg(num_classes=10)

# 3. Loss with label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# 4. Optimizer with weight decay
optimizer = optim.AdamW(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4             # L2 regularization
)

# 5. (Optional) implement Cutout in training loop
def cutout(x, mask_size=50):
    # x: tensor [B,C,H,W]
    B, C, H, W = x.shape
    y = x.clone()
    for i in range(B):
        top = torch.randint(0, H-mask_size, ())
        left = torch.randint(0, W-mask_size, ())
        y[i, :, top:top+mask_size, left:left+mask_size] = 0
    return y

# Example training step
for images, targets in train_loader:
    images = cutout(images, mask_size=60)       # optional Cutout
    optimizer.zero_grad()
    logits = model(images)
    loss = criterion(logits, targets)
    loss.backward()
    optimizer.step()

#### TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

# 1. Data augmentation layer
data_augment = tf.keras.Sequential([
    layers.RandomResizedCrop(224, 224),  # crop & resize
    layers.RandomFlip('horizontal'),     # flip
    layers.RandomRotation(0.1),          # small rotation
    layers.RandomContrast(0.2)           # photometric
])

# 2. Build model with BatchNorm and SpatialDropout
def build_cnn_with_reg(num_classes=10):
    inputs = layers.Input((224,224,3))
    x = data_augment(inputs)                     # augment
    x = layers.Rescaling(1./255)(x)
    # Pretrained backbone
    base = tf.keras.applications.ResNet50(
        include_top=False, weights=None, pooling='avg'
    )
    x = base(x)
    x = layers.BatchNormalization()(x)           # regularizing BN
    x = layers.SpatialDropout2D(0.3)(tf.expand_dims(tf.expand_dims(x,1),1))
    x = layers.Flatten()(x)
    x = layers.Dense(
        num_classes,
        activation='softmax',
        kernel_regularizer=regularizers.l2(1e-4) # weight decay
    )(x)
    return models.Model(inputs, x)

model = build_cnn_with_reg(num_classes=10)

# 3. Compile with label smoothing
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# 4. Early stopping callback
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# 5. Fit
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stop]
)

### Работа с большими датасетами. Даталоадеры и датасеты. Автоматизация препроцессинга

Когда набор данных не помещается в памяти, вам нужен конвейер, который
1. Обеспечит потоковое чтение объектов с диска
2. Автоматически сделает препроцессинг/аугментацию на лету
3. Эффективно организует данные для подачи в нейронную сеть (сформирует батчи и т.д.)

И PyTorch, и TensorFlow предоставляют абстракции для автоматизации этой работы.

#### PyTorch

In [ ]:
# Dataset
# Encodes how to load and transform a single example.
# Must implement __len__ and __getitem__.

import os
from PIL import Image
import torch
from torch.utils.data import Dataset

class ImageFolderDataset(Dataset):
    def __init__(self, root_dir, classes, transform=None):
        self.root_dir = root_dir
        self.classes = classes                         # e.g. ['cat','dog']
        self.transform = transform
        # build list of (image_path, label_idx)
        self.samples = []
        for idx, cls in enumerate(classes):
            cls_dir = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith(('.jpg','.png')):
                    self.samples.append((os.path.join(cls_dir, fname), idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, i):
        path, label = self.samples[i]
        img = Image.open(path).convert('RGB')          # load from disk
        if self.transform:
            img = self.transform(img)                  # augment/preprocess
        return img, label

In [ ]:
# DataLoader
# Wraps the Dataset to provide batching, shuffling, parallel I/O.

# 1. Define transforms: augmentation + normalization
train_transforms = T.Compose([
    T.RandomResizedCrop(224),       # random crop & resize
    T.RandomHorizontalFlip(),       # random flip
    T.ColorJitter(0.2,0.2,0.2,0.1), # random brightness/contrast/sat/hue
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],
                [0.229,0.224,0.225])
])

# 2. Create Dataset
train_ds = ImageFolderDataset(
    root_dir='data/train',
    classes=['cat','dog'],       # example
    transform=train_transforms
)

# 3. Create DataLoader
train_loader = DataLoader(
    train_ds,
    batch_size=64,
    shuffle=True,                # randomize each epoch
    num_workers=4,               # parallel data loading
    pin_memory=True              # speed up GPU transfers
)

# 4. Use in training loop
for images, labels in train_loader:
    images = images.to(device)   # move to GPU
    labels = labels.to(device)
    # forward / backward / optimize...

### TensorFlow

In [ ]:
# Creating the Dataset
# From file paths + labels, or from a directory structure.


import tensorflow as tf

# 1. List file paths & labels
import glob
file_paths = glob.glob('data/train/*/*.jpg')
labels = [0 if 'cat' in p else 1 for p in file_paths]

# 2. Create a tf.data.Dataset from tensors
ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))

In [ ]:
# Mapping: load, decode, augment, batch

AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess(path, label):
    # Read file, decode JPEG, convert to float32
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    # Data augmentation
    img = tf.image.random_crop(img, size=[200,200,3])
    img = tf.image.random_flip_left_right(img)
    # Resize & normalize
    img = tf.image.resize(img, [224,224])
    return img, label

train_ds = (
    ds.shuffle(10000)                    # buffer shuffle
      .map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
      .batch(64)
      .prefetch(AUTOTUNE)                # overlap data prep & model
)

#### Бест практики

1. Parallel I/O:
	* PyTorch → `num_workers>0`
	* TF → `num_parallel_calls=tf.data.AUTOTUNE`
2. Prefetching:
	* PyTorch → `prefetch_factor`
    * TF → `.prefetch()`
3. Caching (for small datasets):
    * TF → `.cache()` для избежания выполнения одних и тех же операций
4. Mixed Augmentation:
    * Выполнение CPU-bound операций (преобразования) в пайплайне датасета
    * GPU-bound операции (цвет, блюр) внутри модели model

### Использования GPU

Зачем использовать ГПУ?
* Массивный параллелизм: Современные GPU имеют тысячи ядер, оптимизированных для выполнения одной и той же арифметической операции над большими массивами (например, матричные умножения, свертки), что дает ускорение на порядки по сравнению с CPU.
* Высокая пропускная способность памяти: GPU гораздо быстрее перемещают данные в память устройства и из нее, уменьшая ботлнек ввода-вывода при обработке больших тензоров.
* Сокращение времени обучения: Более быстрые итерации позволяют больше экспериментировать, использовать большие модели или большие батчи и обучаться за часы, а не за дни.

### PyTorch

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# 1. Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Instantiate model and move to GPU
model = FCNNClassifier(input_dim=20, hidden_dim=50, num_classes=3)
model.to(device)  # all parameters + buffers moved

# 3. Prepare data loader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 4. Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for X, y in train_loader:
        # move batch to GPU
        X = X.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad()
        logits = model(X)         # compute on GPU
        loss = criterion(logits, y)
        loss.backward()           # gradients on GPU
        optimizer.step()

### TensorFlow

In [ ]:
import tensorflow as tf

# 1. List available GPUs
gpus = tf.config.list_physical_devices('GPU')
print("GPUs:", gpus)

# 2. (Optional) Set memory growth to avoid allocating all GPU memory
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 3. Build and compile model as usual
model = build_fcnn_classifier(input_dim=20, hidden_units=50, num_classes=3)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 4. Fit — Keras will automatically use all available GPUs
history = model.fit(train_ds, epochs=10, batch_size=64)